In [231]:
import pymysql
import sqlalchemy
import mysql.connector
from sqlalchemy import create_engine
from sqlalchemy import text
import pandas as pd
import math


bd = "sakila"
connection_string = 'mysql+pymysql://root:' + "Thomas1010??" + '@localhost/'+bd
engine = create_engine(connection_string)


#2

def rentals_month1(year1, month1):

    

    T = '''with temp_table as (
                select rental_id, inventory_id, customer_id, staff_id, date_format(rental_date, '%Y') as year, date_format(rental_date, '%m') as month
                from rental)

            select * from temp_table'''

    add = year + ' and month = ' + month + ';'


    t2 = T + '''
    where year = ''' + add
    

    with engine.connect() as connection:    
        query = text(t2)
        result = connection.execute(query)
    df1 = pd.DataFrame(result.all())

    return df, year, month
    
df1, year1, month1 = rentals_month1("2005", "05")


#3
def rental_count_month1(df1, year1, month1):

    df1 = df1[["customer_id", "rental_id", "year", "month"]]
    
    new_df1 = df1.groupby(["customer_id"]).agg({
        "rental_id" : "count"})
    new_df1 = new_df1.sort_values(by="rental_id", ascending=False)

    name = 'rentals_' + month1 + '_' + year1

    new_df1 = new_df1.rename(columns={"rental_id":name})

    

    return new_df1

new_df1 = rental_count_month1(df1, year1, month1)


#4
#define month 2 to compare

def rentals_month2(year2, month2):

    

    T = '''with temp_table as (
                select rental_id, inventory_id, customer_id, staff_id, date_format(rental_date, '%Y') as year, date_format(rental_date, '%m') as month
                from rental)

            select * from temp_table'''

    add = year2 + ' and month = ' + month2 + ';'


    t2 = T + '''
    where year = ''' + add
    

    with engine.connect() as connection:    
        query = text(t2)
        result = connection.execute(query)
    df2 = pd.DataFrame(result.all())

    return df2, year2, month2
    
df2, year2, month2 = rentals_month2("2005", "06")


#3
def rental_count_month2(df2, year2, month2):

    df2 = df2[["customer_id", "rental_id", "year", "month"]]
    
    new_df2 = df2.groupby(["customer_id"]).agg({
        "rental_id" : "count"})
    new_df2 = new_df2.sort_values(by="rental_id", ascending=False)

    name = 'rentals_' + month2 + '_' + year2

    new_df2 = new_df2.rename(columns={"rental_id":name})

    

    return new_df2

new_df2 = rental_count_month2(df2, year2, month2)

def compare_rentals(new_df1, new_df2):

    final_df = pd.merge(new_df1, new_df2, on='customer_id', how='outer')
           
            
    final_df["rentals_05_2005"] = final_df["rentals_05_2005"].fillna(0)
    final_df["rentals_06_2005"] = final_df["rentals_06_2005"].fillna(0)


    final_df["difference"] = final_df["rentals_05_2005"] - final_df["rentals_06_2005"]

    return final_df
    
compare_rentals(new_df1, new_df2)


,rentals_05_2005,rentals_06_2005,difference
customer_id,,,
1,2.0,7.0,-5.0
2,1.0,1.0,0.0
3,2.0,4.0,-2.0
4,0.0,6.0,-6.0
5,3.0,5.0,-2.0
...,...,...,...
595,1.0,2.0,-1.0
596,6.0,2.0,4.0
597,2.0,3.0,-1.0
